# README: How to train a model on a dataset of Czerny Etudes

In [1]:
import tensorflow as tf

from music_generator.model import MusicModel
from music_generator.serializers.discrete_time_serializer import DiscreteTimeMidiSerializer
import music_generator.utilities.sequence_utils as sequence_utils
import music_generator.utilities.utils as utils

print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.2.0-dev20200327


# Check if CUDA and GPU are working

In [2]:
for message in utils.check_cuda_and_gpu():
    print(message)

CUDA and GPU Available.


# Create model

The default model architecture and hyperparameters are used.

In [3]:
description = 'czerny_etudes'

model = MusicModel(
    ckpt_dir='./training_checkpoints/{}'.format(description),
    log_dir='./logs/{}'.format(description))

model.compile()

model.summary()

Model: "music_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         45568     
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         1312768   
_________________________________________________________________
dropout (Dropout)            (None, None, 512)         0         
_________________________________________________________________
batch_normalization (BatchNo (None, None, 512)         2048      
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 512)         2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 512)         0         
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 512)         

# Create Dataset

A folder of MIDI files are serialized into event sequences.

All sequences are then transposed over a range from 1 whole-step down, to 1.5 whole-steps up. This effectively multiplies the number of sequences by five, and similarly increases training time per epoch by about a factor of six. The result is reduced overfitting and better generated sequences. Whether the benefit outweighs the additional training time will depend on the training data and compositional style.

Finally, the sequences are windowed and the last event split off to use as a label for the model to train on - predicting the next event from the given sequence.

A training and validation dataset are created from the sequences and labels - the validation dataset is used in a callback to reduce the learning rate when validation loss plateaus.

In [4]:
data_path = './training_data/czerny_etudes/'

serializer = DiscreteTimeMidiSerializer()

sequences = serializer.serialize_folder(data_path)
sequences = sequence_utils.transpose(sequences, down=-2, up=3)
sequences, labels = sequence_utils.window(sequences)

dataset_train, dataset_val = sequence_utils.make_tf_datasets(sequences, labels)

Training Sequences: 677621
Validation Sequences: 35665


# Train Model on Dataset

Progress can also be monitored via TensorBoard.

The learning rate of the optimizer will be reduced when validation loss stalls.

The checkpoint with the best training loss will be saved.

In [ ]:
epochs = 100

try:
    history = model.fit(dataset_train,
                    validation_data=dataset_val,
                    epochs=epochs,
                    verbose=1)
except KeyboardInterrupt as e:
    print('\nStopping training...')

Epoch 1/100
5293/5293 [==============================] - 804s 152ms/step - loss: 1.9984 - sparse_categorical_accuracy: 0.5424 - val_loss: 1.0716 - val_sparse_categorical_accuracy: 0.7048 - lr: 3.0000e-04
Epoch 2/100
5293/5293 [==============================] - 816s 154ms/step - loss: 0.9519 - sparse_categorical_accuracy: 0.7301 - val_loss: 0.7542 - val_sparse_categorical_accuracy: 0.7797 - lr: 3.0000e-04
Epoch 3/100
5293/5293 [==============================] - 823s 156ms/step - loss: 0.7016 - sparse_categorical_accuracy: 0.7922 - val_loss: 0.6066 - val_sparse_categorical_accuracy: 0.8176 - lr: 3.0000e-04
Epoch 4/100
5293/5293 [==============================] - 818s 155ms/step - loss: 0.5581 - sparse_categorical_accuracy: 0.8294 - val_loss: 0.5196 - val_sparse_categorical_accuracy: 0.8428 - lr: 3.0000e-04
Epoch 5/100
5293/5293 [==============================] - 820s 155ms/step - loss: 0.4628 - sparse_categorical_accuracy: 0.8556 - val_loss: 0.4685 - val_sparse_categorical_accuracy: 0.85